In [1]:
train_path = './imagesets/B/train.txt'
test_path = './imagesets/B/test.txt'
val_path = './imagesets/B/val.txt'

save_path = '/home/newDisk/tool/carla_dataset_tool/dataset/'

# read imageset(train_path, test_path, val_path)
train_set = set()
test_set = set()
val_set = set()

with open(train_path, 'r') as f:
    lines = f.readlines()
    for line in lines:
        train_set.add(line.replace('\n',''))

with open(test_path, 'r') as f:
    lines = f.readlines()
    for line in lines:
        test_set.add(line.replace('\n',''))

with open(val_path, 'r') as f:
    lines = f.readlines()
    for line in lines:
        val_set.add(line.replace('\n',''))

set_A_05 = ['1226_1700', '1226_1713', '1226_1727', '1226_1741', '1226_1756']
set_B_02 = ['0104_2223', '0104_2309', '0104_2328', '0104_2343', '0105_0013']
set_C_10 = ['0116_0117', '0116_0125', '0116_0132', '0116_0140', '0116_0149']
set_D_06 = ['0104_1949', '0104_2002', '0104_2016', '0104_2032', '0104_2056']

current_set = set_B_02

In [2]:
import os
import shutil
import argparse

proj_path = '/home/newDisk/tool/carla_dataset_tool/'
dir_set = []

for i in range(len(current_set)):
    subset = current_set[i]
    dir = '/home/newDisk/tool/carla_dataset_tool/raw_data/record_2024_'+subset
    dir_set.append(dir)


# get inner frame id, look if both exist label and lidar file
lidar_ckeck = [path+'/vehicle.tesla.model3.master/velodyne/' for path in dir_set]
label_check = [path+'/vehicle.tesla.model3.master/velodyne_semantic/' for path in dir_set]
image_check = [path+'/vehicle.tesla.model3.master/image_2/' for path in dir_set]
calib_path = proj_path + 'test_data/000001.txt'


lidar_target_path = {'train': proj_path+'dataset/training/velodyne/', 'test': proj_path+'dataset/testing/velodyne/'}
image_target_path = {'train': proj_path+'dataset/training/image_2/', 'test': proj_path+'dataset/testing/image_2/'}
label_target_path = {'train': proj_path+'dataset/training/label_2/', 'test': proj_path+'dataset/testing/label_2/'}
calib_target_path = {'train': proj_path+'dataset/training/calib/', 'test': proj_path+'dataset/testing/calib/'}

# create folder if not exist
for key in lidar_target_path.keys():
    if not os.path.exists(lidar_target_path[key]):
        os.makedirs(lidar_target_path[key])
    if not os.path.exists(image_target_path[key]):
        os.makedirs(image_target_path[key])
    if not os.path.exists(label_target_path[key]):
        os.makedirs(label_target_path[key])
    if not os.path.exists(calib_target_path[key]):
        os.makedirs(calib_target_path[key])



for train_frame in train_set:
    lidar_name = '00000' + train_frame[-5:]+'.bin'
    label_name = '00000' + train_frame[-5:]+'.txt'
    image_name = '00000' + train_frame[-5:]+'.png'
    lidar_dir = lidar_ckeck[int(train_frame[0])]
    label_dir = label_check[int(train_frame[0])]
    image_dir = image_check[int(train_frame[0])]
    # check if file exist
    if os.path.exists(image_dir+image_name):
        image_file_path = os.path.join(image_dir, image_name)
    else:
        image_file_path = '/home/newDisk/tool/carla_dataset_tool/raw_data/record_2024_0104_1949/vehicle.tesla.model3.master/image_2/0000012836.png'
    if os.path.exists(lidar_dir+lidar_name) and os.path.exists(label_dir+label_name):
        # copy lidar file
        shutil.copy(os.path.join(lidar_dir, lidar_name), os.path.join(lidar_target_path['train'], train_frame+'.bin'))
        
        # copy label file
        with open(os.path.join(label_dir, label_name), 'r') as f:
            lines = f.readlines()
            lines = lines[:-1]
            fix_lines = ""
            for line in lines:
                elements = line.split(" ")
                # print('This is the test version, when fix the recorder, update here')
                # x, y, z, l, w, h, rot, lab, _, _, _ = elements
                x, y, z, l, w, h, rot, lab, _, _ = elements
                l = abs(round(float(l),4))
                w = abs(round(float(w),4))
                h = abs(round(float(h),4))
                x = round(float(x),4)
                y = round(float(y),4)
                z = round(float(z),4)
                rot = round(float(rot),4)
                # if lab == 'Bus': lab = 'Van'
                # x, y, z, dx, dy, dz, rot, lab, _, _, _ = elements
                rot -= 3.14 if rot > 3.14 else 0
                rot = round(float(rot),4)
                fix_line = "{} {} {} {} {} {} {} {} {} {} {} {} {} {} {}\n".format(lab, '0', '0', '0', '0', '0', '0', '0', h, w, l, x, y, z, rot)
                # fix_line = "{} {} {} {} {} {} {} {}\n".format(x,y,z,dx,dy,dz,rot,lab)
                fix_lines += fix_line
        with open(os.path.join(label_target_path['train'], train_frame+'.txt'), 'w') as f:
            f.writelines(fix_lines)
        
        # copy image file
        shutil.copy(image_file_path, os.path.join(image_target_path['train'], train_frame+'.png'))
    
        # copy calib file
        shutil.copy(calib_path, os.path.join(calib_target_path['train'], train_frame+'.txt'))
    
    else:
        print('file not exist')
        print(train_frame)


file not exist
020570
file not exist
236428
file not exist
022562
file not exist
342437
file not exist
113106
file not exist
428513
file not exist
429479
file not exist
429665
file not exist
342050
file not exist
431366
file not exist
236479
file not exist
428348
file not exist
020732
file not exist
021419
file not exist
429803
file not exist
236443
file not exist
112596
file not exist
237514
file not exist
236170
file not exist
234895
file not exist
342692
file not exist
236128
file not exist
113238
file not exist
022067
file not exist
021059
file not exist
341363
file not exist
236101
file not exist
020609
file not exist
020381
file not exist
430898
file not exist
021077
file not exist
430928
file not exist
114348
file not exist
342572
file not exist
236881
file not exist
021926
file not exist
113091
file not exist
429374
file not exist
236809
file not exist
429887
file not exist
112584
file not exist
342935
file not exist
429818
file not exist
428783
file not exist
234892
file not e

In [3]:
for train_frame in test_set:
    lidar_name = '00000' + train_frame[-5:]+'.bin'
    label_name = '00000' + train_frame[-5:]+'.txt'
    image_name = '00000' + train_frame[-5:]+'.png'
    lidar_dir = lidar_ckeck[int(train_frame[0])]
    label_dir = label_check[int(train_frame[0])]
    image_dir = image_check[int(train_frame[0])]
    # check if file exist
    if os.path.exists(image_dir+image_name):
        image_file_path = os.path.join(image_dir, image_name)
    else:
        image_file_path = '/home/newDisk/tool/carla_dataset_tool/raw_data/record_2024_0104_1949/vehicle.tesla.model3.master/image_2/0000012836.png'
    if os.path.exists(lidar_dir+lidar_name) and os.path.exists(label_dir+label_name):
        # copy lidar file
        shutil.copy(os.path.join(lidar_dir, lidar_name), os.path.join(lidar_target_path['test'], train_frame+'.bin'))
        
        # copy label file
        with open(os.path.join(label_dir, label_name), 'r') as f:
            lines = f.readlines()
            lines = lines[:-1]
            fix_lines = ""
            for line in lines:
                elements = line.split(" ")
                # print('This is the test version, when fix the recorder, update here')
                # x, y, z, l, w, h, rot, lab, _, _, _ = elements
                x, y, z, l, w, h, rot, lab, _, _ = elements
                l = abs(round(float(l),4))
                w = abs(round(float(w),4))
                h = abs(round(float(h),4))
                x = round(float(x),4)
                y = round(float(y),4)
                z = round(float(z),4)
                rot = round(float(rot),4)
                # if lab == 'Bus': lab = 'Van'
                # x, y, z, dx, dy, dz, rot, lab, _, _, _ = elements
                rot -= 3.14 if rot > 3.14 else 0
                rot = round(float(rot),4)
                fix_line = "{} {} {} {} {} {} {} {} {} {} {} {} {} {} {}\n".format(lab, '0', '0', '0', '0', '0', '0', '0', h, w, l, x, y, z, rot)
                # fix_line = "{} {} {} {} {} {} {} {}\n".format(x,y,z,dx,dy,dz,rot,lab)
                fix_lines += fix_line
        with open(os.path.join(label_target_path['test'], train_frame+'.txt'), 'w') as f:
            f.writelines(fix_lines)
        
        # copy image file
        shutil.copy(image_file_path, os.path.join(image_target_path['test'], train_frame+'.png'))
    
        # copy calib file
        shutil.copy(calib_path, os.path.join(calib_target_path['test'], train_frame+'.txt'))
    
    else:
        print('file not exist')
        print(train_frame)

file not exist
341528
file not exist
021740
file not exist
113799
file not exist
114165
file not exist
236311
file not exist
235867
file not exist
234796
file not exist
428381
file not exist
236023
file not exist
235978
file not exist
112857
file not exist
114387
file not exist
431270
file not exist
021413
file not exist
431060
file not exist
428864
file not exist
430421
file not exist
236713
file not exist
022043
file not exist
020558
file not exist
113562
file not exist
342038
file not exist
237556
file not exist
021257
file not exist
236575
file not exist
022697
file not exist
430313
file not exist
342773
file not exist
342134
file not exist
429260
file not exist
022607
file not exist
235078
file not exist
429734
file not exist
020813
file not exist
341048
file not exist
112437
file not exist
020702
file not exist
020594
file not exist
235723
file not exist
022736
file not exist
429080
file not exist
343106
file not exist
342002
file not exist
020630
file not exist
112740
file not e

In [4]:
for train_frame in val_set:
    lidar_name = '00000' + train_frame[-5:]+'.bin'
    label_name = '00000' + train_frame[-5:]+'.txt'
    image_name = '00000' + train_frame[-5:]+'.png'
    lidar_dir = lidar_ckeck[int(train_frame[0])]
    label_dir = label_check[int(train_frame[0])]
    image_dir = image_check[int(train_frame[0])]
    # check if file exist
    if os.path.exists(image_dir+image_name):
        image_file_path = os.path.join(image_dir, image_name)
    else:
        image_file_path = '/home/newDisk/tool/carla_dataset_tool/raw_data/record_2024_0104_1949/vehicle.tesla.model3.master/image_2/0000012836.png'
    if os.path.exists(lidar_dir+lidar_name) and os.path.exists(label_dir+label_name):
        # copy lidar file
        shutil.copy(os.path.join(lidar_dir, lidar_name), os.path.join(lidar_target_path['train'], train_frame+'.bin'))
        
        # copy label file
        with open(os.path.join(label_dir, label_name), 'r') as f:
            lines = f.readlines()
            lines = lines[:-1]
            fix_lines = ""
            for line in lines:
                elements = line.split(" ")
                # print('This is the test version, when fix the recorder, update here')
                # x, y, z, l, w, h, rot, lab, _, _, _ = elements
                x, y, z, l, w, h, rot, lab, _, _ = elements
                l = abs(round(float(l),4))
                w = abs(round(float(w),4))
                h = abs(round(float(h),4))
                x = round(float(x),4)
                y = round(float(y),4)
                z = round(float(z),4)
                rot = round(float(rot),4)
                # if lab == 'Bus': lab = 'Van'
                # x, y, z, dx, dy, dz, rot, lab, _, _, _ = elements
                rot -= 3.14 if rot > 3.14 else 0
                rot = round(float(rot),4)
                fix_line = "{} {} {} {} {} {} {} {} {} {} {} {} {} {} {}\n".format(lab, '0', '0', '0', '0', '0', '0', '0', h, w, l, x, y, z, rot)
                # fix_line = "{} {} {} {} {} {} {} {}\n".format(x,y,z,dx,dy,dz,rot,lab)
                fix_lines += fix_line
        with open(os.path.join(label_target_path['train'], train_frame+'.txt'), 'w') as f:
            f.writelines(fix_lines)
        
        # copy image file
        shutil.copy(image_file_path, os.path.join(image_target_path['train'], train_frame+'.png'))
    
        # copy calib file
        shutil.copy(calib_path, os.path.join(calib_target_path['train'], train_frame+'.txt'))
    
    else:
        print('file not exist')
        print(train_frame)

file not exist
236767
file not exist
235222
file not exist
236413
file not exist
021992
file not exist
341147
file not exist
235060
file not exist
342683
file not exist
430340
file not exist
022682
file not exist
113361
file not exist
235588
file not exist
020489
file not exist
112767
file not exist
235813
file not exist
237337
file not exist
020585
file not exist
343493
file not exist
342548
file not exist
020534
file not exist
343007
file not exist
019976
file not exist
022358
file not exist
341336
file not exist
021548
file not exist
235498
file not exist
021140
file not exist
022409
file not exist
235633
file not exist
236959
file not exist
428624
file not exist
021266
file not exist
114756
file not exist
114417
file not exist
234805
file not exist
429386
file not exist
021638
file not exist
020027
file not exist
115008
file not exist
429206
file not exist
020768
file not exist
113580
file not exist
341057
file not exist
236914
file not exist
021362
file not exist
112569
file not e

In [5]:
# copy imageset

image_set_path = proj_path + 'dataset/ImageSets/'
if not os.path.exists(image_set_path):
    os.makedirs(image_set_path)

shutil.copy(train_path, image_set_path+'train.txt')
shutil.copy(test_path, image_set_path+'test.txt')
shutil.copy(val_path, image_set_path+'val.txt')

'/home/newDisk/tool/carla_dataset_tool/dataset/ImageSets/val.txt'

In [6]:
# deprecated

In [7]:

# # copy lidar
# target_path = {'train': proj_path+'dataset/training/velodyne/', 'test': proj_path+'dataset/testing/velodyne/'}
# for key in target_path:
#     if not os.path.exists(target_path[key]):
#         os.makedirs(target_path[key])
# source_path = []
# for i in dir_set:
#     source_path.append(i+'/vehicle.tesla.model3.master/velodyne/')

# for train_frame in train_set:
#     lidar_name = '00000' + train_frame[-5:]+'.bin'
#     dir = source_path[int(train_frame[0])]
#     file_path = dir+lidar_name
#     shutil.copy(os.path.join(dir, lidar_name), os.path.join(target_path['train'], train_frame+'.bin'))

# for test_frame in test_set:
#     lidar_name = '00000' + test_frame[-5:]+'.bin'
#     dir = source_path[int(test_frame[0])]
#     file_path = dir+lidar_name
#     shutil.copy(os.path.join(dir, lidar_name), os.path.join(target_path['test'], test_frame+'.bin'))

# for val_frame in val_set:
#     lidar_name = '00000' + val_frame[-5:]+'.bin'
#     dir = source_path[int(val_frame[0])]
#     file_path = dir+lidar_name
#     shutil.copy(os.path.join(dir, lidar_name), os.path.join(target_path['train'], val_frame+'.bin'))

In [8]:
# # copy image
# target_path = {'train': proj_path+'dataset/training/image_2/', 'test': proj_path+'dataset/testing/image_2/'}
# for key in target_path:
#     if not os.path.exists(target_path[key]):
#         os.makedirs(target_path[key])
# source_path = []
# for i in dir_set:
#     source_path.append(i+'/vehicle.tesla.model3.master/image_2/')
# for train_frame in train_set:
#     lidar_name = '00000' + train_frame[-5:]+'.png'
#     dir = source_path[int(train_frame[0])]
#     file_path = dir+lidar_name
#     shutil.copy(os.path.join(dir, lidar_name), os.path.join(target_path['train'], train_frame+'.png'))
# for test_frame in test_set:
#     lidar_name = '00000' + test_frame[-5:]+'.png'
#     dir = source_path[int(test_frame[0])]
#     file_path = dir+lidar_name
#     shutil.copy(os.path.join(dir, lidar_name), os.path.join(target_path['test'], test_frame+'.png'))
# for val_frame in val_set:
#     lidar_name = '00000' + val_frame[-5:]+'.png'
#     dir = source_path[int(val_frame[0])]
#     file_path = dir+lidar_name
#     shutil.copy(os.path.join(dir, lidar_name), os.path.join(target_path['train'], val_frame+'.png'))
    